# Import software libraries

In [2]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [3]:
# Connect to SQLite database.
conn = sqlite3.connect('data/prod_sample.db')
conn


In [4]:
# List all the tables in the database.

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Query to list all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])


stock_description
online_retail_history


# Read data from the `online_retail_history` table

In [5]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = 'SELECT * FROM online_retail_history'

# Use the read_sql function in pandas to read a query into a DataFrame.
online_retail_history = pd.read_sql(query, conn)

# Preview the first five rows of the data.
online_retail_history.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [6]:
# Get the shape of the data.
online_retail_history.shape


(15321, 8)

# Read data from the `stock_description` table

In [7]:
# Write the query to be executed that selects everything from the stock_description table.
query = 'SELECT * FROM stock_description'



# Use the read_sql function in pandas to read a query into a DataFrame.
stock_description = pd.read_sql(query, conn)


# Preview the first five rows of the data.
stock_description.head()


,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [8]:
# Get the shape of the data.
stock_description.shape


(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [9]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.
query = '''SELECT left_table.*,
                    right_table.description
            FROM online_retail_history AS left_table
            LEFT JOIN stock_description AS right_table
                ON left_table.stockcode = right_table.stockcode'''


# Use the read_sql function in pandas to read a query into a DataFrame.
history_and_description = pd.read_sql(query, conn)


# Preview the first five rows of the data.
history_and_description.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [10]:
# Get the shape of the data.
history_and_description.shape


(17032, 9)

# Identify and fix corrupt or unusable data

In [11]:
# Check the value counts of the "Description" field.
history_and_description['Description'].value_counts()


CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
?                                     1711
REGENCY CAKESTAND 3 TIER              1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64

In [12]:
# Remove rows where "Description" is just a question mark (?).
history_and_description_filtered = history_and_description[history_and_description['Description'] != '?']


# Preview the first five rows of the data.
history_and_description_filtered.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Identify and remove duplicates

In [13]:
# Identify all duplicated data.
duplicated_data = \
history_and_description_filtered[history_and_description_filtered.duplicated(keep = False)]

print('Number of rows with duplicated data:',
     duplicated_data.shape[0])




Number of rows with duplicated data: 223


In [14]:
# Print the duplicated data.
duplicated_data.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
176,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
178,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
496,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
497,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
569,537955,20725,1,2010-12-09 11:28:00,1.65,u16782,United Kingdom,1.65,LUNCH BAG RED RETROSPOT


In [15]:
# Remove the duplicated data.
history_and_description_final = \
history_and_description_filtered[~history_and_description_filtered.duplicated()]


# Preview the first five rows of the data.
history_and_description_final.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Correct date formats

In [16]:
# Get the data types for every column in the DataFrame.
history_and_description_final.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      15206 non-null  object 
 1   StockCode    15206 non-null  object 
 2   Quantity     15206 non-null  int64  
 3   InvoiceDate  15206 non-null  object 
 4   Price        15194 non-null  float64
 5   CustomerID   12435 non-null  object 
 6   Country      15206 non-null  object 
 7   TotalAmount  15194 non-null  float64
 8   Description  15206 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.2+ MB


In [17]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.
history_and_description_final['InvoiceDate'] = \
pd.to_datetime(history_and_description_final['InvoiceDate'],
              format = '%Y-%m-%d')



In [18]:
# Get the data types for every column in the converted DataFrame.
history_and_description_final.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      15206 non-null  object        
 1   StockCode    15206 non-null  object        
 2   Quantity     15206 non-null  int64         
 3   InvoiceDate  15206 non-null  datetime64[ns]
 4   Price        15194 non-null  float64       
 5   CustomerID   12435 non-null  object        
 6   Country      15206 non-null  object        
 7   TotalAmount  15194 non-null  float64       
 8   Description  15206 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.2+ MB


# Examine the table before finishing

In [19]:
# Preview the first five rows of the data.
history_and_description_final.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [20]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
history_and_description_final.to_pickle('history_and_description_final.pickle')

# Confirm the data were written to a pickle file
pd.read_pickle('history_and_description_final.pickle').head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [21]:
# Close any connections to the database.
conn.close()
